# Student Name: Huynh Truong Tu
 Below is my assignment for Sprint13's "Tensorflow"

-------------------------------------------------------------------------------------------

# Intro to tensorflow

In [37]:
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

In [38]:
a = tf.constant(5)
b = tf.constant(7)
add = tf.add(a, b)
sess = tf.Session()
output = sess.run(add)
print(output) # 12
sess.close()

12


In [39]:
c = tf.placeholder(tf.int32)
d = tf.placeholder(tf.int32)
add = tf.add(c, d)
sess = tf.Session()
output = sess.run(add, feed_dict={c:5, d:7})
print(output) # 12
output = sess.run(add, feed_dict={c:20, d:32})
print(output) # 52
sess.close()


12
52


### NOTE:
Following tensorflow 2 from now on.

# Logistic regress on Tensorflow

In [40]:
import numpy as np
x_train = np.array([[0,0],[0,1],[1,0],[1,1]])
y_train = np.array([[0],[0],[0],[1]])

In [41]:
x = tf.placeholder(tf.float32, [None, 2])
t = tf.placeholder(tf.float32, [None, 1])

In [42]:
W = tf.Variable(tf.zeros([2,1]))
b = tf.Variable(tf.zeros([1]))

In [43]:
y = tf.sigmoid(tf.matmul(x, W) + b)
cross_entropy = tf.reduce_sum(-t * tf.log(y) - (1 - t) * tf.log(1 - y))

In [44]:
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy)

In [45]:
correct_prediction = tf.equal(tf.sign(y - 0.5), tf.sign(t - 0.5))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [46]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [47]:
for epoch in range(1000):
    sess.run(train_step, feed_dict={
        x:x_train,
        t:y_train
    })
# Display the correct answer rate every 100 times
    if epoch % 100 == 0:
        acc_val = sess.run(
            accuracy, feed_dict={
                x:x_train,
                t:y_train})
        print ('epoch: %d, Accuracy: %f'
               %(epoch, acc_val))

epoch: 0, Accuracy: 0.750000
epoch: 100, Accuracy: 1.000000
epoch: 200, Accuracy: 1.000000
epoch: 300, Accuracy: 1.000000
epoch: 400, Accuracy: 1.000000
epoch: 500, Accuracy: 1.000000
epoch: 600, Accuracy: 1.000000
epoch: 700, Accuracy: 1.000000
epoch: 800, Accuracy: 1.000000
epoch: 900, Accuracy: 1.000000


In [48]:
#Check if the learning result is correct
classified = sess.run(correct_prediction, feed_dict={
    x:x_train,
    t:y_train
})
#Check output y
prob = sess.run(y, feed_dict={
    x:x_train,
    t:y_train
})
print(classified)
# [[ True]
# [ True]
# [ True]
# [ True]]

print(prob)
# [[  1.96514215e-04]
# [  4.90498319e-02]
# [  4.90498319e-02]
# [  9.31203783e-01]]

[[ True]
 [ True]
 [ True]
 [ True]]
[[1.9654632e-04]
 [4.9049824e-02]
 [4.9049824e-02]
 [9.3120384e-01]]


In [49]:
print('W:', sess.run(W))
print('b:', sess.run(b))
# W: [[ 5.5699544]
# [ 5.5699544]]
# b: [-8.53457928]

W: [[5.5699544]
 [5.5699544]]
b: [-8.534579]


In [50]:
sess.close()

# Tensorflow (v1)

# Problem 1 - Looking back on the scratch

### Steps to build a ML model
1. Load
2. Define model (class) along with needed stuffs (params, input shapes,..., procedure, batch ...)
3. Init stuff and use the model

# Problem 2 - Correspond between scratch and tensorflow

In [51]:
"""
Binary classification of Iris dataset using neural network implemented in TensorFlow
"""
import pandas as pd
from sklearn.model_selection import train_test_split

#Load dataset
df = pd.read_csv("Iris.csv")

#Condition extraction from data frame
df = df[(df["Species"] == "Iris-versicolor") | (df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
# NumPy 配列に変換
X = np.array(X)
y = np.array(y)
# Convert label to number
y[y == "Iris-versicolor"] = 0
y[y == "Iris-virginica"] = 1
y = y.astype(np.int64)[:, np.newaxis]
#Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
class GetMiniBatch:
    """
    Iterator to get a mini-batch
    Parameters
    ----------
    X : The following forms of ndarray, shape (n_samples, n_features)
      Training data
    y : The following form of ndarray, shape (n_samples, 1)
      Correct answer value
    batch_size : int
      Batch size
    seed : int
      NumPy random number seed
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]
# Hyperparameter settings
learning_rate = 0.001
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1
#Determine the shape of the argument to be passed to the calculation graph
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# train mini batch iterator
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
def example_net(x):
    """
    Simple 3-layer neural network
    """
    tf.random.set_random_seed(0)
    # Declaration of weight and bias
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.add and + are equivalent
    return layer_output
#Read network structure                              
logits = example_net(X)
# Objective function
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
# Optimization method
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# Estimated result
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
#Indicator value calculation
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
#Initialization of variable
init = tf.global_variables_initializer()

#Run calculation graph
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        #Loop for each epoch
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int64)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # Loop for each mini-batch
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})            
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
        total_loss /= n_samples

        val_loss, acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}".format(epoch, total_loss, val_loss, acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))


C:\Users\ADMIN\AppData\Local\Temp/ipykernel_3444/1202908980.py:45: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)


Epoch 0, loss : 26.6982, val_loss : 312.7151, acc : 0.375
Epoch 1, loss : 23.9825, val_loss : 279.9050, acc : 0.375
Epoch 2, loss : 21.3399, val_loss : 248.1234, acc : 0.375
Epoch 3, loss : 18.7862, val_loss : 217.3301, acc : 0.375
Epoch 4, loss : 16.3442, val_loss : 187.9264, acc : 0.375
Epoch 5, loss : 13.9856, val_loss : 159.4701, acc : 0.375
Epoch 6, loss : 11.6769, val_loss : 131.5804, acc : 0.375
Epoch 7, loss : 9.4153, val_loss : 104.1883, acc : 0.375
Epoch 8, loss : 7.2025, val_loss : 77.2788, acc : 0.375
Epoch 9, loss : 5.0426, val_loss : 50.8798, acc : 0.375
test_acc : 0.500


### Steps to build in tensorflow
1. Load dataset
2. Define model and it's flow
   1. Define parameters (variable) and inputs (placeholders)
   2. Define model's prediction (or forward flow)
   3. Define models' loss function
   4. Get an optimizer
   5. Define the traing flow
      - Optimize
      - Training scores
      - Loss
      - Log,...
3. Prepare epoch and batch
4. Initialize paremeters
5.  Start a session
  - Train in batches
  - Use the model and stuss defined in tf syntax
6.  Close session

**Tensorflow 1.0 has a lazy execution approach to it.**

# Problem 3 - Create a model of Iris using all three types of objective variables

## Data Prep

In [52]:
#load dataset
df = pd.read_csv('Iris.csv')

print('shape: ', df.shape)
print(df.head(5))

shape:  (150, 6)
   Id  SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm      Species
0   1            5.1           3.5            1.4           0.2  Iris-setosa
1   2            4.9           3.0            1.4           0.2  Iris-setosa
2   3            4.7           3.2            1.3           0.2  Iris-setosa
3   4            4.6           3.1            1.5           0.2  Iris-setosa
4   5            5.0           3.6            1.4           0.2  Iris-setosa


In [53]:
# predictor
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
print('X shape:', X.shape)
print(X.head(5))

X shape: (150, 4)
   SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm
0            5.1           3.5            1.4           0.2
1            4.9           3.0            1.4           0.2
2            4.7           3.2            1.3           0.2
3            4.6           3.1            1.5           0.2
4            5.0           3.6            1.4           0.2


In [54]:
# label
y = df['Species']
print('y shape: ', y.shape)
print(y.head(5))

y shape:  (150,)
0    Iris-setosa
1    Iris-setosa
2    Iris-setosa
3    Iris-setosa
4    Iris-setosa
Name: Species, dtype: object


In [55]:
# Convert to numpy
X = np.array(X)
y = np.array(y)
print(X.shape, y.shape)

(150, 4) (150,)


In [56]:
# Encode y from text to number label
print('init shape y: ', y.shape)
#label encode
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
y = LabelEncoder().fit_transform(y)
#reshape for onehot use
y = y.reshape(-1,1)
enc = OneHotEncoder()
y = enc.fit_transform(y).toarray()
print('shape: ', y.shape)
print(y[:5])

init shape y:  (150,)
shape:  (150, 3)
[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]


In [57]:
from sklearn.model_selection import train_test_split
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.9)
print('Train/Test size: ', [data.shape for data in [X_train, X_test, y_train, y_test]])
# train validation split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, train_size = 0.8)
print('Train/Test size: ', [data.shape for data in [X_train, X_val, y_train, y_val]])

Train/Test size:  [(135, 4), (15, 4), (135, 3), (15, 3)]
Train/Test size:  [(108, 4), (27, 4), (108, 3), (27, 3)]


## Model Prep

In [58]:
# some variables
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = y_train.shape[1]

print('input nodes: ', n_input)
print('sample size: ', n_samples)
print('prediction classes: ', n_classes)

input nodes:  4
sample size:  108
prediction classes:  3


In [59]:
# train mini batch iterator
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

print('minibatch prepared!')

minibatch prepared!


C:\Users\ADMIN\AppData\Local\Temp/ipykernel_3444/1202908980.py:45: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)


In [60]:
# Hyperparameter settings
learning_rate = 0.001
batch_size = 10
num_epochs = 100
n_hidden1 = 50
n_hidden2 = 100

print('HYPER PARAMS')
print('alpha learning rate: ', learning_rate)
print('batch size: ', batch_size)
print('epoch num', num_epochs)
print('n_hidden1: ', n_hidden1)
print('n_hidden2: ', n_hidden2)

HYPER PARAMS
alpha learning rate:  0.001
batch size:  10
epoch num 100
n_hidden1:  50
n_hidden2:  100


In [61]:
#Determine the shape of the argument to be passed to the calculation graph
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
print('X holder: ',X)
print('Y holder: ', Y)

X holder:  Tensor("Placeholder_14:0", shape=(None, 4), dtype=float32)
Y holder:  Tensor("Placeholder_15:0", shape=(None, 3), dtype=float32)


# Model

### Test layer

In [62]:
# weight
w1 = tf.Variable(tf.random_normal([n_input, n_classes]))
b1 = tf.Variable(tf.random_normal([n_classes]))
print('w:', w1)
print('b:', b1)

w: <tf.Variable 'Variable_18:0' shape=(4, 3) dtype=float32>
b: <tf.Variable 'Variable_19:0' shape=(3,) dtype=float32>


In [63]:
# layer
print('X: ', X)
layer1 = tf.matmul(X, w1)
print('after matrix multiplication:', layer1)
layer1 = tf.add(layer1, b1)
print('after adding bias:', layer1)
layer1 = tf.nn.relu(layer1)
print('after activation: ', layer1)

X:  Tensor("Placeholder_14:0", shape=(None, 4), dtype=float32)
after matrix multiplication: Tensor("MatMul_9:0", shape=(None, 3), dtype=float32)
after adding bias: Tensor("Add_13:0", shape=(None, 3), dtype=float32)
after activation:  Tensor("Relu_5:0", shape=(None, 3), dtype=float32)


In [64]:
session = tf.Session()
def do(stuff, input):
  return session.run(stuff, feed_dict = input)
def prep_session():
  session.run(tf.global_variables_initializer())

prep_session()
forward = do(layer1, {X: X_train, Y: y_train})
print('forward shape: ', forward.shape)
print('forward: ', forward[0])


forward shape:  (108, 3)
forward:  [ 2.8272464  0.        11.3075695]


## Objective Function

In [65]:
logits = layer1 # assuming 1 layer model
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))
print('loss:', loss_op)


# the equivalent or the tf.softmax's implementation!
softmax = tf.nn.softmax(logits)
loss_op2 = -tf.reduce_sum(Y * tf.log(softmax), 1)
loss_op2 = tf.reduce_mean(loss_op2)

loss: Tensor("Mean_8:0", shape=(), dtype=float32)


In [66]:
# test loss op
tf.set_random_seed(0)

x_in, y_in = X_train[:5], y_train[:5]
run = lambda x: do(x, {X: x_in, Y: y_in})
print('y_true: ', y_in)

loss = run(loss_op)
print('loss: ', loss)
loss2 = run(loss_op2)
print('loss2: ', loss2.mean())

y_true:  [[0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]]
loss:  8.445068
loss2:  8.445068


## Prediction

In [67]:
def prediction_function(logits):
  pred = np.argmax(logits, axis = 1).reshape(-1,1)
  return enc.transform(pred).toarray()
logits_ = run(logits)
print('logits: ', logits_)
print('pred: ', prediction_function(logits_))


logits:  [[ 2.8272464  0.        11.3075695]
 [ 4.3492875  0.        12.723299 ]
 [ 3.602052   0.        11.18089  ]
 [ 3.6349647  0.        11.244568 ]
 [ 3.4191263  0.        10.493351 ]]
pred:  [[0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]


## Optimizer

In [68]:
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
print(optimizer)

## Some Metrics

In [69]:
# accuracy
def accuracy_function(pred, real):
  correct_pred = np.argmax(pred,axis = 1) == np.argmax(real,axis = 1)
  return correct_pred.mean()
pred = prediction_function(logits_)
real = y_in
print('accuracy_function: ', accuracy_function(pred,real))

accuracy_function:  0.2


## Training

In [70]:
train_optimizer = optimizer.minimize(loss_op)

def accuracy(logits, y):
  prediction = prediction_function(logits)
  acc = accuracy_function(prediction, y)
  return acc

def train_batch(mini_batch_x, mini_batch_y):
  input_xy = {X: mini_batch_x, Y: mini_batch_y}
  run = lambda x: do(x, input_xy)
  # train
  run(train_optimizer)
  # loss
  batch_loss= run(loss_op)
  # accuracy
  batch_acc = accuracy(run(logits), mini_batch_y)
  return batch_loss, batch_acc

def train_epoch(X_val, y_val, iterator, verbose= False):
  #Loop for each epoch
  total_loss = 0
  total_acc = 0
  # train
  for i, (mini_batch_x, mini_batch_y) in enumerate(iterator):
      batch_loss, batch_acc = train_batch(mini_batch_x, mini_batch_y)
      total_loss += batch_loss
      total_acc += batch_acc * batch_size
      if verbose:
        print("Batch: {}, b_loss: {:.3f}, b_acc: {:.3f}".format(i, batch_loss, batch_acc))
  total_loss /= n_samples
  total_acc /= n_samples

  input_xy = {X: X_val, Y: y_val}
  run = lambda x: do(x, input_xy)
  val_loss = run(loss_op)
  val_acc = accuracy(run(logits), y_val)
  return total_loss, total_acc, val_loss, val_acc

def train(X_train, y_train, X_val, y_val, epoch = 10, verbose_interval = 10):
  iterator = GetMiniBatch(X_train, y_train, batch_size=batch_size)
  for i in range(epoch):
    # print('Starting epoch: ', i)
    total_loss, total_acc, val_loss, val_acc = train_epoch(X_val, y_val, iterator)
    if (i+1)%verbose_interval == 0 or i == 0:
      print("Epoch {} Result -- tot_loss : {:.2f}, tot_acc : {:.2f}, val_loss : {:.2f}, val_acc : {:.2f}".format(i, total_loss, total_acc, val_loss, val_acc))

    # print('______')

session.close()
session = tf.Session()
prep_session()
train(X_train, y_train, X_val, y_val, epoch = 100, verbose_interval = 10)

C:\Users\ADMIN\AppData\Local\Temp/ipykernel_3444/1202908980.py:45: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)


Epoch 0 Result -- tot_loss : 0.57, tot_acc : 0.36, val_loss : 5.98, val_acc : 0.30
Epoch 9 Result -- tot_loss : 0.46, tot_acc : 0.36, val_loss : 4.70, val_acc : 0.30
Epoch 19 Result -- tot_loss : 0.34, tot_acc : 0.36, val_loss : 3.50, val_acc : 0.30
Epoch 29 Result -- tot_loss : 0.29, tot_acc : 0.37, val_loss : 2.91, val_acc : 0.26
Epoch 39 Result -- tot_loss : 0.25, tot_acc : 0.40, val_loss : 2.47, val_acc : 0.41
Epoch 49 Result -- tot_loss : 0.21, tot_acc : 0.56, val_loss : 2.08, val_acc : 0.44
Epoch 59 Result -- tot_loss : 0.17, tot_acc : 0.62, val_loss : 1.70, val_acc : 0.56
Epoch 69 Result -- tot_loss : 0.13, tot_acc : 0.66, val_loss : 1.34, val_acc : 0.59
Epoch 79 Result -- tot_loss : 0.11, tot_acc : 0.66, val_loss : 1.05, val_acc : 0.63
Epoch 89 Result -- tot_loss : 0.09, tot_acc : 0.67, val_loss : 0.91, val_acc : 0.67
Epoch 99 Result -- tot_loss : 0.09, tot_acc : 0.67, val_loss : 0.84, val_acc : 0.63


# Problem 4 - Creating a model of house prices

In [71]:
# data set
house_price_pd = pd.read_csv('./houseprice_train.csv')
house_price_pd.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [72]:
# selection
selected = house_price_pd[['GrLivArea', 'YearBuilt','SalePrice']]
print('selected: ', selected.shape, selected.columns)
print(selected.head())

selected:  (1460, 3) Index(['GrLivArea', 'YearBuilt', 'SalePrice'], dtype='object')
   GrLivArea  YearBuilt  SalePrice
0       1710       2003     208500
1       1262       1976     181500
2       1786       2001     223500
3       1717       1915     140000
4       2198       2000     250000


In [73]:
# check nan
print(selected.isna().sum(axis = 0))

GrLivArea    0
YearBuilt    0
SalePrice    0
dtype: int64


In [74]:
# normalize
print(selected.describe().transpose()[['mean', 'std']])

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled = scaler.fit_transform(selected)
print(pd.DataFrame(scaled).describe().transpose()[['mean', 'std']])

explainatory = scaled[:,:-1]
objective = scaled[:,-1].reshape(-1,1)
print(explainatory.shape, explainatory[:3])
print(objective.shape, objective[:3])

                    mean           std
GrLivArea    1515.463699    525.480383
YearBuilt    1971.267808     30.202904
SalePrice  180921.195890  79442.502883
           mean       std
0 -1.277517e-16  1.000343
1  1.046347e-15  1.000343
2  1.362685e-16  1.000343
(1460, 2) [[ 0.37033344  1.05099379]
 [-0.48251191  0.15673371]
 [ 0.51501256  0.9847523 ]]
(1460, 1) [[0.34727322]
 [0.00728832]
 [0.53615372]]


In [75]:
# train test split
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(explainatory, objective, train_size = 0.8)
print('train shape: ', [i.shape for i in [X_train, X_val, y_train, y_val]])

train shape:  [(1168, 2), (292, 2), (1168, 1), (292, 1)]


In [76]:
# prep vars
input_dimention = X_train.shape[-1]
batch_size = 20
lr = 0.1


In [77]:
# placeholder
X = tf.placeholder("float", [None, input_dimention])
Y = tf.placeholder("float", [None, 1])
print(X,Y)

Tensor("Placeholder_16:0", shape=(None, 2), dtype=float32) Tensor("Placeholder_17:0", shape=(None, 1), dtype=float32)


In [78]:
# model
w = tf.Variable(tf.random_normal([input_dimention, 1]))
b = tf.Variable(tf.random_normal([1]))
linear = tf.matmul(X, w)
linear = tf.add(linear, b)

model = linear

In [79]:
# loss
loss = Y - model
loss = tf.math.square(loss)
loss = tf.math.reduce_mean(loss) / 2

# optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr)
optimize_operation = optimizer.minimize(loss)

In [80]:
# train
def _train(X_train, y_train, X_val, y_val):
  input_xy = {X: X_train, Y: y_train}
  run = lambda x: do(x, input_xy)
  # train
  run(optimize_operation)
  # predict
  train_loss =  run(loss)
  input_xy = {X: X_val, Y: y_val}
  val_loss = run(loss)
  return train_loss, val_loss

def train(X_train, y_train, X_val, y_val, epoch = 100, print_interval = 10):
  print(X_train.shape)
  for i in range(epoch): 
    train_loss, val_loss = _train(X_train, y_train, X_val, y_val)
    if (i + 1) % print_interval == 0 or i == 0:
      print(f'Ilteration: {i}, SSE_train: {train_loss}, SSE_val: {val_loss}')

session.close()
session = tf.Session()
prep_session()
train(X_train, y_train, X_val, y_val)

(1168, 2)
Ilteration: 0, SSE_train: 2.559596538543701, SSE_val: 2.71960186958313
Ilteration: 9, SSE_train: 0.559020459651947, SSE_val: 0.6395063996315002
Ilteration: 19, SSE_train: 0.2301533818244934, SSE_val: 0.2698774039745331
Ilteration: 29, SSE_train: 0.17963533103466034, SSE_val: 0.2065722644329071
Ilteration: 39, SSE_train: 0.17091837525367737, SSE_val: 0.19398769736289978
Ilteration: 49, SSE_train: 0.16932176053524017, SSE_val: 0.19117355346679688
Ilteration: 59, SSE_train: 0.16902121901512146, SSE_val: 0.19046470522880554
Ilteration: 69, SSE_train: 0.16896399855613708, SSE_val: 0.190261110663414
Ilteration: 79, SSE_train: 0.16895300149917603, SSE_val: 0.1901945322751999
Ilteration: 89, SSE_train: 0.1689509153366089, SSE_val: 0.1901702731847763
Ilteration: 99, SSE_train: 0.16895049810409546, SSE_val: 0.19016072154045105


# Problem 5 - Creating a MNIST model

In [81]:
#data set
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

#random subset since the data is too large
from sklearn.model_selection import train_test_split
X_train, _, y_train, _ = train_test_split(X_train, y_train, train_size=0.075)

#reshape and add one channel to X
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

#scaling
X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255
#one hot encode for multiclass labels!
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', sparse=True)
y_train_one_hot = enc.fit_transform(y_train[:, np.newaxis]).toarray()
y_test_one_hot = enc.transform(y_test[:, np.newaxis])

#validation split
x_train, x_val, y_train, y_val = train_test_split(X_train, y_train_one_hot, train_size=0.8)
print('train/val')
print([i.shape for i in [x_train, x_val, y_train, y_val]])

train/val
[(3600, 28, 28, 1), (900, 28, 28, 1), (3600, 10), (900, 10)]


C:\Users\ADMIN\AppData\Local\Temp/ipykernel_3444/417275906.py:14: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X_train = X_train.astype(np.float)
C:\Users\ADMIN\AppData\Local\Temp/ipykernel_3444/417275906.py:15: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X_test = X_test.astype(np.float)


In [82]:
print('label sample: \n', y_train[1])

label sample: 
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]


In [83]:
print(X_train.shape)
# prep vars
input_shape = X_train.shape[1:]
print('input_shape: ', input_shape)
output_shape = y_train.shape[1:]
print('output_shape: ', output_shape)
n_classes = output_shape[0]
print('n_classes: ', n_classes)
init_in_channels = input_shape[-1]
print('init_in_channels: ', init_in_channels)

# Place holders
X = tf.placeholder('float', [None, *input_shape])
Y = tf.placeholder('float', [None, *output_shape])
print(X,Y)

(4500, 28, 28, 1)
input_shape:  (28, 28, 1)
output_shape:  (10,)
n_classes:  10
init_in_channels:  1
Tensor("Placeholder_18:0", shape=(None, 28, 28, 1), dtype=float32) Tensor("Placeholder_19:0", shape=(None, 10), dtype=float32)


### Model
VGG16

In [84]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

def conv2d(x, W, b, strides=1):
    # print(f'Making conv2d -- in:{x.shape}, weight: {W.shape}, bias: {b.shape}')
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],padding='SAME')

def make_var(shape, name = 'LongVar'):
  return tf.Variable(tf.random_normal([*shape]), name = name)
def relu(input_):
  return tf.nn.relu(input_)

def VGG16(): #structure of VGG16
  weight_biases = {

    # !1
    'conv1.1' : [
      make_var(shape = [3,3, init_in_channels,64], name = 'w1'),
      make_var(shape = [64], name = 'b1')
    ],

    # !2
    'conv2.1' : [
      make_var(shape = [3,3, 64,128], name = 'w2'),
      make_var(shape = [128], name = 'b2')
    ],

    # !3
    'conv3.1' : [
      make_var(shape = [3,3, 128,256], name = 'w3'),
      make_var(shape = [256], name = 'b3')
    ],
  }

  # first 
  first = conv2d(X, *weight_biases['conv1.1'])
  firstpool = maxpool2d(first, k = 2)
  # second
  second = conv2d(firstpool, *weight_biases['conv2.1'])
  secondpool = maxpool2d(second, k = 2)
  # # third
  third = conv2d(secondpool, *weight_biases['conv3.1'])
  thirdpool = maxpool2d(third, k = 2)

  # flatten
  last_conv_layer = thirdpool
  flatted_shape = np.array(last_conv_layer.shape[1:]).prod()
  flatten = tf.reshape(last_conv_layer, [-1, flatted_shape])
  #fcs
  fc1 = tf.layers.dense(flatten, units = 4096, activation = 'relu')
  fc2 = tf.layers.dense(fc1, units = n_classes, activation = 'softmax')

  result = fc2
  layers = [first, firstpool, second, secondpool, third, thirdpool, flatten, fc1, fc2]
  verbose = True
  if verbose: 
    print('---------------------------------------')
    for layer in layers:
      if verbose: print(layer)
    print('---------------------------------------')
  return result

model = VGG16()
logits = model

#loss
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))
#opt
optimizer = tf.train.AdamOptimizer(learning_rate=lr)
train_optimizer = optimizer.minimize(loss_op)
#accuracy
accuracy = tf.math.reduce_mean(tf.keras.metrics.categorical_accuracy(logits, Y))

C:\Users\ADMIN\AppData\Local\Temp/ipykernel_3444/2029434593.py:56: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  fc1 = tf.layers.dense(flatten, units = 4096, activation = 'relu')
C:\Users\ADMIN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\keras\legacy_tf_layers\core.py:255: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  return layer.apply(inputs)
C:\Users\ADMIN\AppData\Local\Temp/ipykernel_3444/2029434593.py:57: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  fc2 = tf.layers.dense(fc1, units = n_classes, activation = 'softmax')


---------------------------------------
Tensor("Relu_6:0", shape=(None, 28, 28, 64), dtype=float32)
Tensor("MaxPool:0", shape=(None, 14, 14, 64), dtype=float32)
Tensor("Relu_7:0", shape=(None, 14, 14, 128), dtype=float32)
Tensor("MaxPool_1:0", shape=(None, 7, 7, 128), dtype=float32)
Tensor("Relu_8:0", shape=(None, 7, 7, 256), dtype=float32)
Tensor("MaxPool_2:0", shape=(None, 4, 4, 256), dtype=float32)
Tensor("Reshape:0", shape=(None, 4096), dtype=float32)
Tensor("dense/Relu:0", shape=(None, 4096), dtype=float32)
Tensor("dense_1/Softmax:0", shape=(None, 10), dtype=float32)
---------------------------------------


In [85]:
#Run calculation graph
with tf.Session() as sess:
  X_train, y_train, X_val, y_val = x_train, y_train, x_val, y_val 
  print([x.shape for x in [X_train, y_train, X_val, y_val]])
  sess.run(tf.initialize_all_variables())
  get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=20)
  for epoch in range(10):
    #Loop for each epoch
    total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int64)
    total_loss = 0
    total_acc = 0
    for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
        # Loop for each mini-batch
        sess.run(train_optimizer, feed_dict={X: mini_batch_x, Y: mini_batch_y})            
        loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
        total_loss += loss
    total_loss /= n_samples

    val_loss, acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
    print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}".format(epoch, total_loss, val_loss, acc))
  # test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
  # print("test_acc : {:.3f}".format(test_acc))

# 15680,10

[(3600, 28, 28, 1), (3600, 10), (900, 28, 28, 1), (900, 10)]
Instructions for updating:
Use `tf.global_variables_initializer` instead.


C:\Users\ADMIN\AppData\Local\Temp/ipykernel_3444/1202908980.py:45: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)


Epoch 0, loss : 3.9329, val_loss : 2.3789, acc : 0.082
Epoch 1, loss : 3.9329, val_loss : 2.3789, acc : 0.082
Epoch 2, loss : 3.9329, val_loss : 2.3789, acc : 0.082
Epoch 3, loss : 3.9329, val_loss : 2.3789, acc : 0.082
Epoch 4, loss : 3.9329, val_loss : 2.3789, acc : 0.082
Epoch 5, loss : 3.9329, val_loss : 2.3789, acc : 0.082
Epoch 6, loss : 3.9329, val_loss : 2.3789, acc : 0.082
Epoch 7, loss : 3.9329, val_loss : 2.3789, acc : 0.082
Epoch 8, loss : 3.9329, val_loss : 2.3789, acc : 0.082
Epoch 9, loss : 3.9329, val_loss : 2.3789, acc : 0.082


-------------------------------------------------------------------------------------------

This is the end of my assignment. Thank you for reading!